# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import csv
import time
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image



#if tf.__version__ < '1.4.0':
#  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
print('done')

c:\users\berger\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


done


## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [72]:
ROOT = os.getcwd()
# What model to download.
MODEL_PATH = 'Model/500frozen_inference_graph.pb'
#Path to input images
PATH_TO_TEST_IMAGES = 'Images/'

PATH_TO_TRUE_LABELS = 'Images/test_labels.csv'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'labelmap.pbtxt')

OUTPUT_PATH = 'Images/'
threshold = 0.1

## Load a (frozen) Tensorflow model into memory.

In [73]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(MODEL_PATH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
print('done')

done


## Helper code

In [74]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [75]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
TEST_IMAGE_PATHS = []
for file in os.listdir(PATH_TO_TEST_IMAGES):
    if file.endswith(".jpg"):
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES, file))
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'newImages{}.jpg'.format(i)) for i in range(1, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (1024, 1024)
print(TEST_IMAGE_PATHS)

['Images/00002.jpg', 'Images/00016.jpg', 'Images/00021.jpg', 'Images/00023.jpg', 'Images/00037.jpg', 'Images/00039.jpg', 'Images/00043.jpg', 'Images/00051.jpg', 'Images/00058 - Copy.jpg', 'Images/00058.jpg', 'Images/00059.jpg', 'Images/00067.jpg', 'Images/00095.jpg', 'Images/00113.jpg', 'Images/00120.jpg', 'Images/00128.jpg', 'Images/0013.jpg', 'Images/00139.jpg', 'Images/0014.jpg', 'Images/00146.jpg', 'Images/0021.jpg', 'Images/0026.jpg', 'Images/0028.jpg', 'Images/0034.jpg', 'Images/0036.jpg', 'Images/0044.jpg', 'Images/0048.jpg', 'Images/0055.jpg', 'Images/0064.jpg', 'Images/0077.jpg', 'Images/0081.jpg', 'Images/0084.jpg', 'Images/0089.jpg', 'Images/0110.jpg', 'Images/0111.jpg', 'Images/0119.jpg', 'Images/0127.jpg', 'Images/0129.jpg', 'Images/0143.jpg', 'Images/0145.jpg', 'Images/0157.jpg', 'Images/0159.jpg', 'Images/0161.jpg', 'Images/0169.jpg', 'Images/0175.jpg', 'Images/0179.jpg', 'Images/0182.jpg', 'Images/0203.jpg', 'Images/0213.jpg', 'Images/0217.jpg', 'Images/0221.jpg', 'Imag

In [76]:
configCPU = tf.ConfigProto(device_count={'GPU': 0})
configCPU.gpu_options.allow_growth = False
configCPU.log_device_placement=False

configGPU = tf.ConfigProto()
configGPU.gpu_options.allow_growth = True
configGPU.log_device_placement=False

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session(config=configGPU) as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})
      
    
      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
        
      
      #saver.save(sess,graphOutputTest)
  return output_dict
print('done')

def run_inference_for_multiple_image(images, graph):
  with graph.as_default():
    with tf.Session(config=configGPU) as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      for image_path in images:
          print(image_path)
          image = Image.open(image_path)
          image_np = load_image_into_numpy_array(image)
          image_np_expanded = np.expand_dims(image_np, axis=0)
          # Run inference
          output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})
      
    
      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
        
      
      #saver.save(sess,graphOutputTest)
  return output_dict
print('done')

done
done


In [77]:
def get_intersect(pred_coords,true_coords):
    true_coords=[float(i) for i in true_coords]
    pred_coords=[float(i)*1024 for i in pred_coords]
    predicted_surface = (pred_coords[2]-pred_coords[0])*(pred_coords[3]-pred_coords[1])
    true_surface = (true_coords[2]-true_coords[0])*(true_coords[3]-true_coords[1])
    
    intersect_ymin = max(pred_coords[1], true_coords[0])
    intersect_xmin = max(pred_coords[0], true_coords[1])
    intersect_ymax = min(pred_coords[3], true_coords[2])
    intersect_xmax = min(pred_coords[2], true_coords[3])
    
    intersect_surface = max(0,  intersect_ymax-intersect_ymin ) * max(0, intersect_xmax - intersect_xmin)
    return intersect_surface/float(predicted_surface+true_surface-intersect_surface)
    

In [78]:
def draw_rect(img_name,xmin,ymin,xmax,ymax,color):
    image =Image.open(PATH_TO_TEST_IMAGES+img_name)
    im = np.array(image, dtype=np.uint8)
    cv2.rectangle(im,(xmax,ymax),(xmin,ymin),color,2)
    img = Image.new("RGB",image.size)
    img = Image.fromarray(im, 'RGB')
    img.save(OUTPUT_PATH+img_name)

In [79]:
def save_img():
    image =Image.open(PATH_TO_TEST_IMAGES+img_name)
    im = np.array(image, dtype=np.uint8)

In [80]:
def run_single_inference():
    output =[]
    for image_path in TEST_IMAGE_PATHS:
      print(image_path)
      #image = misc.imread(image_path)
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.

      output_dict = run_inference_for_single_image(image_np, detection_graph)
      
      #OUTPUT THE COORDINATES OF GOOD PICTURES
      x = output_dict['detection_scores']
      boxes = np.array
      boxes = output_dict['detection_boxes'][x>threshold]
      x = x[x>threshold]
      for score,box in zip(x,boxes):
          output.append([image_path,score,box])
    
    return output

def run_inference():
    output_dict = run_inference_for_multiple_image(TEST_IMAGE_PATHS, detection_graph)



In [81]:

start_time = time.clock()
# run_inference()
print('start time: '+str(start_time))
print('end time: '+str(time.clock()))
print('run took: '+str(time.clock()-start_time))

start time: 6653.892506484468
end time: 6653.892737908033
run took: 0.00029218078452686314


In [82]:
def get_color(score):
    return (int)(255 - 255 * score ), (int)(score * 255 ), 0
    

In [84]:
start_time = time.clock()
pred_list = []
pred_list =run_single_inference()
end_time = time.clock()

Images/00002.jpg
Images/00016.jpg
Images/00021.jpg
Images/00023.jpg
Images/00037.jpg
Images/00039.jpg
Images/00043.jpg
Images/00051.jpg
Images/00058 - Copy.jpg
Images/00058.jpg
Images/00059.jpg
Images/00067.jpg
Images/00095.jpg
Images/00113.jpg
Images/00120.jpg
Images/00128.jpg
Images/0013.jpg
Images/00139.jpg
Images/0014.jpg
Images/00146.jpg
Images/0021.jpg
Images/0026.jpg
Images/0028.jpg
Images/0034.jpg
Images/0036.jpg
Images/0044.jpg
Images/0048.jpg
Images/0055.jpg
Images/0064.jpg
Images/0077.jpg
Images/0081.jpg
Images/0084.jpg
Images/0089.jpg
Images/0110.jpg
Images/0111.jpg
Images/0119.jpg
Images/0127.jpg
Images/0129.jpg
Images/0143.jpg
Images/0145.jpg
Images/0157.jpg
Images/0159.jpg
Images/0161.jpg
Images/0169.jpg
Images/0175.jpg
Images/0179.jpg
Images/0182.jpg
Images/0203.jpg
Images/0213.jpg
Images/0217.jpg
Images/0221.jpg
Images/0223.jpg
Images/0233.jpg
Images/0236.jpg
Images/0243.jpg
Images/0253.jpg
Images/0257.jpg
Images/0264.jpg
Images/0269.jpg
Images/0271.jpg
Images/0272.jpg

In [85]:
x = []
IoU = []
total = 0
items = 0
FN =0
with open(PATH_TO_TRUE_LABELS) as a:
    true_reader=csv.reader(a,delimiter=',');
    for true_row in true_reader:

        temp_IoU = 0
        for pred_row in pred_list:
            if true_row[0]==pred_row[0].split('/')[len(pred_row[0].split('/'))-1]:
                temp_IoU = max(temp_IoU, get_intersect(pred_row[2],true_row[4:]))
        if true_row[0]!='filename':
            if temp_IoU<0.1 :
                draw_rect(true_row[0],int(true_row[4]),int(true_row[5]),int(true_row[6]),int(true_row[7]),(255,255,255))
                FN+=1
            else:
                draw_rect(true_row[0],int(true_row[4]),int(true_row[5]),int(true_row[6]),int(true_row[7]),get_color(pred_row[1]))
        total+=temp_IoU
        items+=1

print('Run took: {}'.format(end_time-start_time))
print('Average time per image: {}'.format((end_time-start_time)/items))
print('IoU is {}'.format(total/items))
print('Number of items: {} with: {} FN and {} TP'.format(items,FN,items-FN))
#GPU Run took 16 seconds (2,67 seconds each)
#CPU RUN TOOK 28 SECONDS (4,67 seconds each)

# Run took: 427.77529488584605
# Average time per image: 2.027371065809697
# IoU is 0.5972971029283503
# Number of items: 211 with: 24 FN and 187 TP

Run took: 343.0629338978524
Average time per image: 1.6258906819803431
IoU is 0.6657786036650453
Number of items: 211 with: 25 FN and 186 TP


In [ ]:

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(get_available_gpus())